**Packages installation**

In [1]:
!pip install forex-python

**Imports**

In [2]:
import numpy as np 
import pandas as pd 

import seaborn as sns 
import matplotlib.pyplot as plt

from forex_python.converter import CurrencyRates
import datetime as dt

Merging all the CSV's into one dataset, and shuffling it with a specified seed for later reproductibility.

In [3]:
dataset = pd.concat([pd.read_csv('Volkswagen_Golf.csv', na_values='Unknown'), pd.read_csv('BMW_Seria 3.csv', na_values='Unknown'), pd.read_csv('Opel_Corsa.csv', na_values='Unknown')])
dataset = dataset.sample(frac=1, random_state=42)
dataset.head()

,brand,model,generation,eng_cap,prod_year,power,fuel_type,car_body,mileage,color,condition,transmission,origin,price,source
1550,Volkswagen,Golf,VII (2012-2020),1 984 cm3,2017,310 KM,Benzyna,Kompakt,77 000 km,Biały,Używane,Automatyczna,NaN,114 000PLN,OTOMOTO
1221,Volkswagen,Golf,VIII (2020-),1 968 cm3,2022,150 KM,Diesel,Kompakt,32 000 km,Szary,Używane,Automatyczna,Polska,95 000PLN,OTOMOTO
1941,Volkswagen,Golf,VIII (2020-),1 968 cm3,2020,150 KM,Diesel,Kompakt,76 020 km,Czarny,Używane,Automatyczna,Francja,86 900PLN,OTOMOTO
573,BMW,Seria 3,E90/E91/E92/E93 (2005-2012),2 996 cm3,2005,258 KM,Benzyna,Sedan,288 858 km,Szary,Używane,Manualna,NaN,32 500PLN,OTOMOTO
173,Opel,Corsa,D (2006-2014),1 229 cm3,2009,80 KM,Benzyna,Kompakt,152 000 km,Czarny,Używane,Manualna,NaN,18 500PLN,OTOMOTO


Checking how the data looks:

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6867 entries, 1550 to 860
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         6867 non-null   object
 1   model         6867 non-null   object
 2   generation    6805 non-null   object
 3   eng_cap       6782 non-null   object
 4   prod_year     6867 non-null   int64 
 5   power         6861 non-null   object
 6   fuel_type     6867 non-null   object
 7   car_body      6867 non-null   object
 8   mileage       6867 non-null   object
 9   color         6867 non-null   object
 10  condition     6867 non-null   object
 11  transmission  6865 non-null   object
 12  origin        4154 non-null   object
 13  price         6867 non-null   object
 14  source        6867 non-null   object
dtypes: int64(1), object(14)
memory usage: 858.4+ KB


As we can see, there are columns, e.g. mileage or price, which definitely should be an integer value, and instead, every column except prod_year is the type of an object. In order to execute exploratory data analysis, we have to prepare the data.

Let's preprocess the data in the following way:

brand - object

model - object

generation - object

eng_cap - int [cm^3]

prod_year - int

power - int [KM]

fuel_type - object

car_body - object

mileage - int [km]

color - object

condition - object

transmission - object

origin - object

price - int [zł]

source - object (we can drop it right away, because in all samples it has one value: **OTOMOTO**)



In [5]:
dataset['eng_cap'] = dataset['eng_cap'].apply(lambda x: float(x.replace('cm3', '').replace(' ', '')) if not pd.isnull(x) else x)
dataset['power'] = dataset['power'].apply(lambda x: float(x.replace('KM', '').replace(' ', '')) if not pd.isnull(x) else x)
dataset['mileage'] = dataset['mileage'].apply(lambda x: int(x.replace('km', '').replace(' ', '')) if not pd.isnull(x) else x)

With the price, situation is more complicated, because it can be given in different currencies. We have to convert them all to PLN

In [6]:
from forex_python.converter import CurrencyRates
import re
import functools

# caching, so that getting online info about certain currency happens only once for the same input 
@functools.cache
def get_currency(base_cur: str, dest_cur: str, date_obj: dt.datetime) -> float:
    c = CurrencyRates()
    try:
        rate = c.get_rate(base_cur=base_cur, dest_cur='PLN', date_obj=date_obj)
    except:
        print('LOOLOZ BLAD:', base_cur, dest_cur)
    if not isinstance(rate, float):
        raise ValueError(f'One of the following function arguments was incorrect: {base_cur, dest_cur, date_obj}')
    return rate

def currency_to_PLN(value: str) -> float:
    pattern = re.compile(r'([\d\s,]+)([^\d\s]+)')
    match = re.search(pattern, value)
    if match is None:
        raise ValueError(f'Unexpected price format: {value}')
    float_val = float(match.group(1).replace(' ', '').replace(',', '.'))
    base_curr = match.group(2)
    date = dt.datetime(2024, 5, 2, 12, 00) # set datetime for reproductability
    rate = get_currency(base_curr, 'PLN', date)
    return round(float_val * rate, 2)

dataset['price'] = dataset['price'].apply(lambda x: float(currency_to_PLN(x)) if not pd.isnull(x) else x)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6867 entries, 1550 to 860
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         6867 non-null   object 
 1   model         6867 non-null   object 
 2   generation    6805 non-null   object 
 3   eng_cap       6782 non-null   float64
 4   prod_year     6867 non-null   int64  
 5   power         6861 non-null   float64
 6   fuel_type     6867 non-null   object 
 7   car_body      6867 non-null   object 
 8   mileage       6867 non-null   int64  
 9   color         6867 non-null   object 
 10  condition     6867 non-null   object 
 11  transmission  6865 non-null   object 
 12  origin        4154 non-null   object 
 13  price         6867 non-null   float64
 14  source        6867 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 858.4+ KB


In [8]:
# dropping unnecessary source column
dataset.drop('source', axis=1, inplace=True)
dataset.head()

,brand,model,generation,eng_cap,prod_year,power,fuel_type,car_body,mileage,color,condition,transmission,origin,price
1550,Volkswagen,Golf,VII (2012-2020),1984.0,2017,310.0,Benzyna,Kompakt,77000,Biały,Używane,Automatyczna,NaN,114000.0
1221,Volkswagen,Golf,VIII (2020-),1968.0,2022,150.0,Diesel,Kompakt,32000,Szary,Używane,Automatyczna,Polska,95000.0
1941,Volkswagen,Golf,VIII (2020-),1968.0,2020,150.0,Diesel,Kompakt,76020,Czarny,Używane,Automatyczna,Francja,86900.0
573,BMW,Seria 3,E90/E91/E92/E93 (2005-2012),2996.0,2005,258.0,Benzyna,Sedan,288858,Szary,Używane,Manualna,NaN,32500.0
173,Opel,Corsa,D (2006-2014),1229.0,2009,80.0,Benzyna,Kompakt,152000,Czarny,Używane,Manualna,NaN,18500.0


In [9]:
def show_missing_values():
    nulls = dataset.isnull()
    null_values = nulls.sum().sort_values(ascending=False)
    percentage = nulls.sum() / nulls.count()*100
    missing_values = pd.concat([null_values, percentage], axis=1, keys=['Number', 'Percentage'], sort=False)
    return missing_values[missing_values['Percentage'] != 0]

show_missing_values()

,Number,Percentage
origin,2713,39.507791
eng_cap,85,1.237804
generation,62,0.902869
power,6,0.087374
transmission,2,0.029125


As we can see, about 40% of the samples don't have their **origin country** specified. After checking (later) how strong correlation between this decision variable and the price is, we will decide whether we drop this column or not.

The rest of the columns have almost no missing values.

Now, we have to preprocess the whole data, i.e. 

In [4]:
from pandas_profiling import ProfileReport
np.

df = pd.DataFrame(np.random.rand(100, 5), columns=['a', 'b', 'c', 'd', 'e'])
profile = ProfileReport(df, title='Test profile report', html={'style':{'full_width':True}})


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.7/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.7/u/import-error